In [6]:
import numpy as np
import cv2

from os import listdir
from os.path import isfile, join

test_set_files = [join('testset', f) for f in listdir('testset') if isfile(join('testset', f))]
train_set_files = [join('trainset', f) for f in listdir('trainset') if isfile(join('trainset', f))]

def compute_greyscale_32x32_descryptors(files):
    descryptors = []
    for img_path in files:
        img = cv2.imread(img_path, 0)
        dsc = cv2.resize(img, (32, 32)).astype(np.float32)
        descryptors.append([dsc, img_path])
    return descryptors

def prepare_data_PCA(dsc_32x32_with_filepaths):
    data_matrix = np.zeros((len(dsc_32x32_with_filepaths), 32*32), np.float32)
    for i in range(0, len(dsc_32x32_with_filepaths)):
        data_matrix[i,:] = dsc_32x32_with_filepaths[i][0].flatten()
    filenames = np.array(dsc_32x32_with_filepaths)[...,1]
    return data_matrix, filenames

def compute_PCA(data_matrix, max_components):
    mean, eigenvectors = cv2.PCACompute(data_matrix, maxComponents=max_components)
    return mean, eigenvectors

def compute_PCA_descryptors(mean, eigenvectors, data_matrix, filenames):
    pc_data_matrix = cv2.PCAProject(data_matrix, mean, eigenvectors)
    descryptors = zip(pc_data_matrix, filenames)
    return descryptors

test_set_32x32_descryptors = compute_greyscale_32x32_descryptors(test_set_files)
train_set_32x32_descryptors = compute_greyscale_32x32_descryptors(train_set_files)

train_data_matrix, train_filenames = prepare_data_PCA(train_set_32x32_descryptors)
mean, eigenvectors = compute_PCA(train_data_matrix, 19)
train_set_PCA_descryptors = compute_PCA_descryptors(mean, eigenvectors, train_data_matrix, train_filenames)

test_data_matrix, test_filenames = prepare_data_PCA(test_set_32x32_descryptors)
test_set_PCA_descryptors = compute_PCA_descryptors(mean, eigenvectors, test_data_matrix, test_filenames)

In [7]:
import pickle

with open('output/descryptors.txt', 'w') as f:
    pickle.dump(
        [
            test_set_32x32_descryptors,
            train_set_32x32_descryptors,
            train_set_PCA_descryptors,
            test_set_PCA_descryptors
        ], f)